In [ ]:
import os

import numpy as np
import pandas as pd
from scipy.optimize import minimize_scalar, minimize
from scipy.stats import poisson, norm, gamma
from scipy.special import factorial
import statsmodels.api as sm

from sklearn.metrics import mean_squared_error

%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
DATASET_PATH = 'datasets/motorbike_ambulance_calls.csv'
FIGURES_PATH = 'figures'

pd.plotting.register_matplotlib_converters()
sns.set_style('whitegrid')
sns.set_palette('muted')
year_palette = {
    2011: 'C0',
    2012: 'C3'
}
season_order = ['winter', 'spring', 'summer', 'autumn']

In [ ]:
motorbike_data = (
    pd.read_csv(
        DATASET_PATH,
        parse_dates=['date'],
        dayfirst=False,
    )
    .set_index('index')
    
    .assign(
        yr=lambda x: np.where(
            x['yr'] == 0,
            2011,
            2012
        )
    )
    .assign(
        season=lambda x: (
            pd.Categorical(
                x['season'], 
                categories=season_order, 
                ordered=True
            )
        )
    )
)
motorbike_data.info()
motorbike_data.head()

In [ ]:
if not os.path.exists(FIGURES_PATH):
    os.mkdir(FIGURES_PATH)

# Categorical and date-time features

In [ ]:
g = sns.relplot(
    x='mnth',
    y='cnt',
    hue='yr',
    legend='full',
    kind='line',
    marker='o',
    aspect=2,
    palette=year_palette,
    data=motorbike_data
)
g._legend.texts[0].set_text('Year')
g.set(
    xlabel='Month', 
    ylabel='Average number of calls', 
)
plt.subplots_adjust(top=0.9)
g.fig.suptitle('Number of calls by month and year')
g.fig.savefig(os.path.join(FIGURES_PATH, 'calls-by-month.png'))

In [ ]:
data_by_year = (
    motorbike_data
    .groupby(['yr', 'mnth'], as_index=False)
    ['cnt'].mean()
)

data_11 = (
    data_by_year
    .loc[lambda x: x['yr'] == 2011, 'cnt']
    .reset_index(drop=True)
)

data_12 = (
    data_by_year
    .loc[lambda x: x['yr'] == 2012, 'cnt']
    .reset_index(drop=True)
)

def years_diff_loss(intercept):
    return mean_squared_error(data_11 + intercept, data_12)

optimum = minimize_scalar(years_diff_loss)
assert optimum.success
print(optimum)

In [ ]:
g = sns.relplot(
    x='mnth',
    y='cnt',
    hue='yr',
    legend='full',
    kind='line',
    marker='o',
    aspect=2,
    palette=year_palette,
    data=(
        motorbike_data
        .assign(cnt=lambda x: np.where(
            x['yr'] == 2011,
            x['cnt'] + optimum.x,
            x['cnt']
        ))
    )
)
g._legend.texts[0].set_text('Year')
g.set(
    xlabel='Month', 
    ylabel='Average number of calls', 
)
plt.subplots_adjust(top=0.9)
g.fig.suptitle('Seasonality change of number of calls by month and year')
g.fig.savefig(os.path.join(FIGURES_PATH, 'seasonality-change-calls-by-month.png'))

In [ ]:
g = sns.relplot(
    x='season',
    y='cnt',
    hue='yr',
    legend='full',
    kind='line',
    marker='o',
    aspect=2,
    palette=year_palette,
    data=motorbike_data
)
g._legend.texts[0].set_text('Year')
g.set(
    xlabel='Season', 
    ylabel='Average number of calls', 
)
plt.subplots_adjust(top=0.9)
g.fig.suptitle('Number of calls by season and year')
g.fig.savefig(os.path.join(FIGURES_PATH, 'calls-by-sesason.png'))

In [ ]:
g = sns.catplot(
    x='season',
    y='cnt',
    hue='yr',
    legend='full',
    kind='bar',
    palette=year_palette,
    aspect=2,
    data=(
        motorbike_data
        .groupby(['yr', 'season'], as_index=False)
        ['cnt']
        .count()
    )
)
g._legend.set_title('Year')
g.set(
    xlabel='Season', 
    ylabel='Number of observations', 
)
plt.subplots_adjust(top=0.9)
g.fig.suptitle('Number of observations by season and year')
g.fig.savefig(os.path.join(FIGURES_PATH, 'observations-by-season.png'))

In [ ]:
g = sns.relplot(
    x='hr',
    y='cnt',
    hue='yr',
    row='workingday',
    legend='full',
    kind='line',
    marker='o',
    aspect=2,
    palette=year_palette,
    data=motorbike_data
)
g._legend.texts[0].set_text('Year')
g.set(
    xlabel='Hour', 
    ylabel='Average number of calls',
    xticks=np.arange(0, 24)
)
for ax in g.axes.flatten():
    ax.tick_params(labelbottom=True)
plt.subplots_adjust(top=0.9, hspace=0.15)
g.fig.suptitle('Number of calls by hour and year')
g.fig.savefig(os.path.join(FIGURES_PATH, 'calls-by-hour.png'))

In [ ]:
g = sns.catplot(
    x='hr',
    y='cnt',
    hue='yr',
    legend='full',
    kind='bar',
    palette=year_palette,
    aspect=2,
    data=(
        motorbike_data
        .groupby(['yr', 'hr'], as_index=False)
        ['cnt']
        .count()
    )
)
g._legend.set_title('Year')
g.set(
    xlabel='Hour', 
    ylabel='Number of observations', 
)
plt.subplots_adjust(top=0.9)
g.fig.suptitle('Number of observations by hour and year')
g.fig.savefig(os.path.join(FIGURES_PATH, 'observations-by-hour.png'))

In [ ]:
g = sns.FacetGrid(
    motorbike_data, 
    col='yr',
    hue='yr',
    palette=year_palette,
    height=8
)
g.map(
    sns.pointplot, 
    'season', 
    'cnt', 
    order=season_order
)
g.axes[0][0].set(
    title='Year 2011',
    xlabel='Season',
    ylabel='Average number of calls'
)
g.axes[0][1].set(
    title='Year 2012',
    xlabel='Season'
)
plt.subplots_adjust(top=0.9)
g.fig.suptitle('Number of calls by season and year')
g.fig.savefig(os.path.join(FIGURES_PATH, 'calls-by-season-significance.png'))

In [ ]:
g = sns.FacetGrid(
    motorbike_data, 
    col='yr',
    hue='yr',
    palette=year_palette,
    height=8
)
g.map(
    sns.pointplot, 
    'mnth', 
    'cnt',
    order=np.arange(1, 13)
)
g.axes[0][0].set(
    title='Year 2011',
    xlabel='Month',
    ylabel='Average number of calls'
)

g.axes[0][1].set(
    title='Year 2012',
    xlabel='Month'
)
plt.subplots_adjust(top=0.9)
g.fig.suptitle('Number of calls by month and year')
g.fig.savefig(os.path.join(FIGURES_PATH, 'calls-by-month-significance.png'))

In [ ]:
g = sns.relplot(
    x='weathersit',
    y='cnt',
    hue='yr',
    legend='full',
    kind='line',
    marker='o',
    aspect=2,
    palette=year_palette,
    data=motorbike_data
)
g._legend.texts[0].set_text('Year')
g.set(
    xlabel='Weather situatuion', 
    ylabel='Average number of calls',
    xticks=[1, 2, 3, 4]
)
plt.subplots_adjust(top=0.9)
g.fig.suptitle('Number of calls by weather situation and year')
g.fig.savefig(os.path.join(FIGURES_PATH, 'calls-by-weathersit.png'))

In [ ]:
g = sns.catplot(
    x='weathersit',
    y='cnt',
    hue='yr',
    legend='full',
    kind='bar',
    palette=year_palette,
    aspect=2,
    data=(
        motorbike_data
        .groupby(['yr', 'weathersit'], as_index=False)
        ['cnt']
        .count()
    )
)
g._legend.set_title('Year')
g.set(
    xlabel='Weather situation', 
    ylabel='Number of observations',
)
plt.subplots_adjust(top=0.9)
g.fig.suptitle('Number of observations by weather situation and year')
g.fig.savefig(os.path.join(FIGURES_PATH, 'observations-by-weathersit.png'))

(
    motorbike_data
    .groupby(['yr', 'weathersit'], as_index=False)
    ['cnt']
    .count()
)

In [ ]:
g = sns.relplot(
    x='hr',
    y='cnt',
    hue='yr',
    row='weekday',
    col='holiday',
    legend='full',
    kind='line',
    marker='o',
    aspect=2,
    palette=year_palette,
    data=motorbike_data
)
g._legend.texts[0].set_text('Year')
g.set(
    xlabel='Hour', 
    ylabel='Average number of calls',
    xticks=np.arange(0, 24)
)
for ax in g.axes.flatten():
    ax.tick_params(labelbottom=True)
plt.subplots_adjust(top=0.95, hspace=0.15)
g.fig.suptitle('Number of calls by hour and year, split by weekday X holiday')
g.fig.savefig(os.path.join(FIGURES_PATH, 'calls-by-hour-by-weekday-by-holiday.png'))

# Numerical features

In [ ]:
numerical_features = {
    'cnt', 'temp', 'atemp', 'hum', 'windspeed'
}

In [ ]:
(
    motorbike_data
    .reindex(columns=numerical_features)
    .corr()
)

In [ ]:
print('Max correlation for each feature:')
(
    motorbike_data
    .reindex(columns=numerical_features)
    .corr()
    .pipe(lambda x: x.subtract(np.diag([1.0] * len(x.columns))))
    .apply(
        lambda x: pd.Series({
            'feature': x.abs().idxmax(), 
            'corr': x[x.abs().idxmax()]
        }), 
        axis='columns'
    )
)

In [ ]:
g = sns.pairplot(
    motorbike_data,
    palette=year_palette,
    kind='reg',
    diag_kind='kde',
    vars=numerical_features
)

plt.subplots_adjust(top=0.9)
g.fig.suptitle('Relations between numerical features')
g.fig.savefig(os.path.join(FIGURES_PATH, 'relations-numerical.png'))

# Target feature distribution

In [ ]:
plt.figure(figsize=(15, 5))
ax = sns.distplot(
    motorbike_data['cnt'],
)
ax.set(
    xlabel='Number of calls'
)
ax.figure.suptitle('Distribution of the number of calls')
ax.figure.savefig(os.path.join(FIGURES_PATH, 'cnt-distribution.png'))

In [ ]:
plt.figure(figsize=(15, 5))
ax = sns.distplot(
    np.log(motorbike_data['cnt']),
)
ax.set(
    xlabel='Log number of calls'
)
ax.figure.suptitle('Distribution of log of the number of calls')
ax.figure.savefig(os.path.join(FIGURES_PATH, 'log-cnt-distribution.png'))

In [ ]:
plt.figure(figsize=(15, 5))
ax = sns.distplot(
    np.sqrt(motorbike_data['cnt']),
)
ax.set(
    xlabel='Square root of number of calls'
)
ax.figure.suptitle('Distribution of square root of the number of calls')
ax.figure.savefig(os.path.join(FIGURES_PATH, 'sqrt-cnt-distribution.png'))

In [ ]:
def neg_poisson_log_likelihood(data, lamb):
    likelihoods = poisson.pmf(data, lamb)
    likelihoods = likelihoods[likelihoods != 0]
    return -np.sum(np.log(likelihoods))

def mle_poisson(data):
    optimum = minimize(
        lambda params, data: neg_poisson_log_likelihood(data, params[0]),
        x0=np.ones(1),
        args=(data,),
        method='SLSQP'
    )
    print(optimum)
    assert optimum.success
    return optimum.x[0]

mle_poisson(np.random.poisson(lam=0.2, size=1000))
mle_poisson(np.random.poisson(lam=7.3, size=1000))

In [ ]:
x_plot = np.arange(1000)
fig = plt.figure(figsize=(15, 5))
sns.distplot(motorbike_data['cnt'])
plt.plot(
    x_plot,
    poisson.pmf(x_plot, mle_poisson(motorbike_data['cnt']))
)

fig.suptitle('Attempt to fit Poisson distribution with MLE')
fig.savefig(os.path.join(FIGURES_PATH, 'fitted-poisson.png'))

In [ ]:
def neg_norm_log_likelihood(data, loc, scale):
    likelihoods = norm.pdf(data, loc=loc, scale=scale)
    likelihoods = likelihoods[likelihoods != 0]
    return -np.sum(np.log(likelihoods))

def mle_norm(data):
    optimum = minimize(
        lambda params, data: neg_norm_log_likelihood(
            data, 
            loc=params[0], 
            scale=params[1]
        ),
        x0=np.array([0, 1]),
        args=(data,),
        method='SLSQP'
    )
    print(optimum)
    assert optimum.success
    return optimum.x

mle_norm(np.random.normal(loc=3, scale=2, size=1000))
mle_norm(np.random.normal(loc=-2, scale=0.3, size=1000))

In [ ]:
mle_norm(motorbike_data['cnt'])

In [ ]:
fig = plt.figure(figsize=(15, 5))
ax = sns.distplot(
    motorbike_data['cnt'], kde=False,
    fit=norm
)
ax.set(
    xlabel='Number of calls'
)
ax.figure.suptitle('Fitted normal distribution')
ax.figure.savefig(os.path.join(FIGURES_PATH, 'fitted-normal.png'))

In [ ]:
fig = plt.figure(figsize=(15, 5))
ax = sns.distplot(
    motorbike_data['cnt'], kde=False,
    fit=gamma
)
ax.set(
    xlabel='Number of calls'
)
ax.figure.suptitle('Fitted gamma distribution')
ax.figure.savefig(os.path.join(FIGURES_PATH, 'fitted-gamma.png'))

# Missing data

In [ ]:
(
    motorbike_data
    ['date']
    .apply(['min', 'max'])
)

In [ ]:
observations_by_hour = (
    motorbike_data
    .groupby(['date', 'hr']).count()
    ['cnt']
    .reindex(pd.MultiIndex.from_product(
        [
            pd.date_range('2011-01-01', '2012-12-31'),
            np.arange(24)
        ],
        names=['date', 'hr']
    ))
)

missing_observations = (
    observations_by_hour
    [lambda x: pd.isnull(x)]
    .reset_index()
    .drop(columns='cnt')
)

missing_observations

In [ ]:
(
    missing_observations
    .assign(yr=lambda x: x['date'].dt.year)
    .groupby('yr')
    .count()
)

In [ ]:
(
    observations_by_hour
    .reset_index()
    .fillna({'cnt': 0})
    .assign(group_mask=lambda x: (x['cnt'] != x['cnt'].shift()).cumsum())
    .groupby(['cnt', 'group_mask'], as_index=False, sort=False)
    .count()
    .loc[lambda x: x['cnt'] == 0, 'hr']
    .value_counts()
)

In [ ]:
missing_periods_distribution = (
    pd.DataFrame(observations_by_hour)
    .fillna(0)
    .assign(group_mask=lambda x: (x != x.shift()).cumsum())
    [lambda x: x['cnt'] == 0]
    .groupby('group_mask', sort=False)
    .count()
    ['cnt']
)

fig = plt.figure(figsize=(15, 5))
sns.distplot(
    missing_periods_distribution, 
    kde=False
)
fig.suptitle('Distribution of periods of missing data')
fig.savefig(os.path.join(FIGURES_PATH, 'missing-periods.png'))

missing_periods_distribution.value_counts().sort_index()